<a href="https://www.kaggle.com/code/larsmagnusson/workshop-21-october?scriptVersionId=147323954" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import load_iris

# Load Iris dataset from sklearn
iris = load_iris(as_frame=True)
iris['data'].head()
#iris['target'].head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
from sklearn.model_selection import train_test_split

# Split data into training and test data using stratified sampling
iris_train_data, iris_test_data, iris_train_targets, iris_test_targets = train_test_split(iris['data'], iris['target'], test_size=0.2, stratify=iris['target'], random_state=1)
iris_train_targets

21     0
47     0
141    2
90     1
64     1
      ..
41     0
93     1
26     0
3      0
42     0
Name: target, Length: 120, dtype: int64

In [4]:
from sklearn.cluster import KMeans

# Create kmeans with 3 clusters and fit to data
kmeans = KMeans(n_clusters=3)
kmeans.fit(iris_train_data)

KMeans(n_clusters=3)

In [5]:
from sklearn.cluster import AgglomerativeClustering

# Create agglomerative with 3 clusters and fit to data
agglomerative = AgglomerativeClustering(n_clusters=3)
agglomerative.fit(iris_train_data)

AgglomerativeClustering(n_clusters=3)

In [6]:
from sklearn.metrics import silhouette_score

# Calculate and show the silhouette coefficient score for kmeans clusterings. kmeans.labels_ gives
# us access to the labeling of the observations. We could also use predict for kmeans and 
# all other inductive clustering algorithms
kmeans_silhouette = silhouette_score(iris_train_data, kmeans.labels_)
kmeans_silhouette

0.5644685864087499

In [7]:
# Calculate and show the silhouette coefficient score for kmeans clusterings. Agglomerative clustering
# is a transductive algorithm, meaning we do not have the possibility to use predict
agglomerative_silhouette = silhouette_score(iris_train_data, agglomerative.labels_)
agglomerative_silhouette

0.5102289970143681

In [8]:
from sklearn.metrics import rand_score

# Calculate the rand index (unadjusted) for both kmeans an agglomerative for the training data
kmeans_rand = rand_score(kmeans.predict(iris_train_data), iris_train_targets)
agglomerative_rand = rand_score(agglomerative.labels_, iris_train_targets)

# Show the scores
(kmeans_rand, agglomerative_rand)

(0.8857142857142857, 0.8705882352941177)

In [9]:
# Calculate and show the rand score for kmeans on test data
kmeans_rand = rand_score(kmeans.predict(iris_test_data), iris_test_targets)
kmeans_rand

0.8528735632183908

In [10]:
# This is a setup to do test data evaluation with transductive clustering algorithms. There might be an easier
# way of doing this, but the point is that you have to retrain for all predictions since there exist no induced
# model we can use to predict.

# We start by creating an empty list for storing the predictions
agglomerative_predictions = []
# We iterate through all the test data instances
for i in range(len(iris_test_targets)):
    # Here we create a new dataframe containing the training data and the current test instance
    augmented_data = iris_train_data.append(iris_test_data.iloc[[i]])
    
    # Create and fit an agglomerative clustering
    agglo = AgglomerativeClustering(n_clusters=3)
    agglo.fit(augmented_data)
    
    # Fetch the label of the last instance in the augmented dataframe and store it in the predictions list
    agglomerative_predictions.append(agglo.labels_[-1])

# Calculate and show the rand score for agglomerative clustering
agglomerative_rand = rand_score(agglomerative_predictions, iris_test_targets)
agglomerative_rand

0.7908045977011494

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


outer_cv = StratifiedKFold(n_splits=5,shuffle=True)
inner_cv = StratifiedKFold(n_splits=4,shuffle=True)

kmeans_params ={}
kmeans_cv = GridSearchCV(KMeans(n_clusters=3), kmeans_params, cv=inner_cv)
scores = cross_val_score(kmeans_cv, iris_train_data, iris_train_targets)
scores.mean()

-12.96704021669674